# 📅 Day 3: Level 1 — Binary Classification (Benign vs Attack)
## 4 Models: Decision Tree, Random Forest, XGBoost GPU, LightGBM GPU

---

**Steps:**
1. Load preprocessed data
2. Train 4 models (DT, RF, XGBoost GPU, LightGBM GPU)
3. Evaluate: Accuracy, F1, Precision, Recall, ROC-AUC, Confusion Matrix
4. Compare all models
5. ROC Curves

---

In [ ]:
import os
os.add_dll_directory(r'C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v13.1\bin\x64')

import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score,
                             roc_auc_score, classification_report, confusion_matrix,
                             roc_curve, auc)
import matplotlib.pyplot as plt
import seaborn as sns
import time
import gc
import json
import joblib
from datetime import datetime

plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

os.makedirs('models', exist_ok=True)
os.makedirs('figures', exist_ok=True)

print(f"✅ Ready | {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 XGBoost: {xgb.__version__} | LightGBM: {lgb.__version__}")

## 📥 Load Preprocessed Data

In [ ]:
print("📥 Loading preprocessed data...")
t0 = time.time()

X_train = np.load('processed/X_train.npy')
X_test = np.load('processed/X_test.npy')
y_train = np.load('processed/y_binary_train.npy')
y_test = np.load('processed/y_binary_test.npy')

with open('processed/preprocessing_metadata.json', 'r') as f:
    meta = json.load(f)
feature_names = meta['feature_names']

# Class balance
n_benign_train = (y_train == 0).sum()
n_attack_train = (y_train == 1).sum()
scale_pos_weight = n_benign_train / n_attack_train if n_attack_train > 0 else 1.0

print(f"✅ Loaded in {time.time()-t0:.1f}s")
print(f"   X_train: {X_train.shape} | X_test: {X_test.shape}")
print(f"   Train — Benign: {n_benign_train:,} | Attack: {n_attack_train:,}")
print(f"   scale_pos_weight: {scale_pos_weight:.4f}")

## 🏋️ Train 4 Models

In [ ]:
# Storage for results
results = {}
models = {}
predictions = {}
probabilities = {}

In [ ]:
# ============================================================
# Model 1: Decision Tree
# ============================================================
print("="*60)
print("🌳 Model 1: Decision Tree")
print("="*60)

dt = DecisionTreeClassifier(
    class_weight='balanced',
    max_depth=15,
    random_state=42
)

t0 = time.time()
dt.fit(X_train, y_train)
train_time_dt = time.time() - t0

t0 = time.time()
y_pred_dt = dt.predict(X_test)
y_prob_dt = dt.predict_proba(X_test)[:, 1]
infer_time_dt = time.time() - t0

models['Decision Tree'] = dt
predictions['Decision Tree'] = y_pred_dt
probabilities['Decision Tree'] = y_prob_dt

print(f"   ⏱️ Train: {train_time_dt:.1f}s | Inference: {infer_time_dt:.1f}s")
print(f"   ✅ Accuracy: {accuracy_score(y_test, y_pred_dt)*100:.4f}%")
print(f"   🎯 F1: {f1_score(y_test, y_pred_dt)*100:.4f}%")

joblib.dump(dt, 'models/binary_dt.joblib')
print("   💾 Saved to models/binary_dt.joblib")

In [ ]:
# ============================================================
# Model 2: Random Forest
# ============================================================
print("="*60)
print("🌲 Model 2: Random Forest")
print("="*60)

rf = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced',
    max_depth=15,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

t0 = time.time()
rf.fit(X_train, y_train)
train_time_rf = time.time() - t0

t0 = time.time()
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]
infer_time_rf = time.time() - t0

models['Random Forest'] = rf
predictions['Random Forest'] = y_pred_rf
probabilities['Random Forest'] = y_prob_rf

print(f"   ⏱️ Train: {train_time_rf:.1f}s | Inference: {infer_time_rf:.1f}s")
print(f"   ✅ Accuracy: {accuracy_score(y_test, y_pred_rf)*100:.4f}%")
print(f"   🎯 F1: {f1_score(y_test, y_pred_rf)*100:.4f}%")

joblib.dump(rf, 'models/binary_rf.joblib')
print("   💾 Saved to models/binary_rf.joblib")

In [ ]:
# ============================================================
# Model 3: XGBoost GPU 🎮
# ============================================================
print("="*60)
print("🎮 Model 3: XGBoost GPU (CUDA)")
print("="*60)

dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_names)
dtest = xgb.DMatrix(X_test, label=y_test, feature_names=feature_names)

xgb_params = {
    'tree_method': 'hist',
    'device': 'cuda',            # 🎮 GPU
    'objective': 'binary:logistic',
    'eval_metric': ['logloss', 'auc', 'error'],
    'max_depth': 8,
    'learning_rate': 0.1,
    'min_child_weight': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'scale_pos_weight': scale_pos_weight,
    'verbosity': 1,
    'seed': 42
}

evals_result_xgb = {}
t0 = time.time()
bst_xgb = xgb.train(
    xgb_params, dtrain,
    num_boost_round=300,
    evals=[(dtrain, 'train'), (dtest, 'test')],
    early_stopping_rounds=20,
    evals_result=evals_result_xgb,
    verbose_eval=50
)
train_time_xgb = time.time() - t0

t0 = time.time()
y_prob_xgb = bst_xgb.predict(dtest, iteration_range=(0, bst_xgb.best_iteration + 1))
y_pred_xgb = (y_prob_xgb > 0.5).astype(int)
infer_time_xgb = time.time() - t0

models['XGBoost GPU'] = bst_xgb
predictions['XGBoost GPU'] = y_pred_xgb
probabilities['XGBoost GPU'] = y_prob_xgb

print(f"\n   🎮 Trained on GPU (CUDA)")
print(f"   ⏱️ Train: {train_time_xgb:.1f}s | Inference: {infer_time_xgb:.1f}s")
print(f"   🏆 Best iteration: {bst_xgb.best_iteration}")
print(f"   ✅ Accuracy: {accuracy_score(y_test, y_pred_xgb)*100:.4f}%")
print(f"   🎯 F1: {f1_score(y_test, y_pred_xgb)*100:.4f}%")

bst_xgb.save_model('models/binary_xgb_gpu.json')
print("   💾 Saved to models/binary_xgb_gpu.json")

del dtrain
gc.collect()

In [ ]:
# ============================================================
# Model 4: LightGBM GPU 🎮
# ============================================================
print("="*60)
print("🎮 Model 4: LightGBM GPU")
print("="*60)

lgb_train = lgb.Dataset(X_train, label=y_train, feature_name=feature_names, free_raw_data=False)
lgb_test = lgb.Dataset(X_test, label=y_test, feature_name=feature_names, reference=lgb_train, free_raw_data=False)

lgb_params = {
    'objective': 'binary',
    'metric': ['binary_logloss', 'auc', 'binary_error'],
    'device': 'gpu',             # 🎮 GPU
    'gpu_use_dp': False,
    'is_unbalance': True,
    'max_depth': 8,
    'learning_rate': 0.1,
    'num_leaves': 127,
    'min_child_samples': 50,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'verbosity': 1,
    'seed': 42,
    'n_jobs': -1
}

evals_result_lgb = {}
callbacks = [
    lgb.log_evaluation(period=50),
    lgb.early_stopping(stopping_rounds=20),
    lgb.record_evaluation(evals_result_lgb)
]

t0 = time.time()
bst_lgb = lgb.train(
    lgb_params,
    lgb_train,
    num_boost_round=300,
    valid_sets=[lgb_train, lgb_test],
    valid_names=['train', 'test'],
    callbacks=callbacks
)
train_time_lgb = time.time() - t0

t0 = time.time()
y_prob_lgb = bst_lgb.predict(X_test, num_iteration=bst_lgb.best_iteration)
y_pred_lgb = (y_prob_lgb > 0.5).astype(int)
infer_time_lgb = time.time() - t0

models['LightGBM GPU'] = bst_lgb
predictions['LightGBM GPU'] = y_pred_lgb
probabilities['LightGBM GPU'] = y_prob_lgb

print(f"\n   🎮 Trained on GPU")
print(f"   ⏱️ Train: {train_time_lgb:.1f}s | Inference: {infer_time_lgb:.1f}s")
print(f"   🏆 Best iteration: {bst_lgb.best_iteration}")
print(f"   ✅ Accuracy: {accuracy_score(y_test, y_pred_lgb)*100:.4f}%")
print(f"   🎯 F1: {f1_score(y_test, y_pred_lgb)*100:.4f}%")

bst_lgb.save_model('models/binary_lgb_gpu.txt')
print("   💾 Saved to models/binary_lgb_gpu.txt")

## 📊 Compare All Models

In [ ]:
# Build comparison table
print("\n" + "="*80)
print("📊 BINARY CLASSIFICATION RESULTS — ALL MODELS")
print("="*80)

model_names = ['Decision Tree', 'Random Forest', 'XGBoost GPU', 'LightGBM GPU']
train_times = [train_time_dt, train_time_rf, train_time_xgb, train_time_lgb]
infer_times = [infer_time_dt, infer_time_rf, infer_time_xgb, infer_time_lgb]

comparison_rows = []
for name in model_names:
    y_p = predictions[name]
    y_pr = probabilities[name]
    row = {
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_p) * 100,
        'F1': f1_score(y_test, y_p, average='binary') * 100,
        'Precision': precision_score(y_test, y_p) * 100,
        'Recall': recall_score(y_test, y_p) * 100,
        'ROC-AUC': roc_auc_score(y_test, y_pr) * 100,
        'Train Time (s)': train_times[model_names.index(name)],
        'Infer Time (s)': infer_times[model_names.index(name)]
    }
    comparison_rows.append(row)

comparison_df = pd.DataFrame(comparison_rows)
comparison_df = comparison_df.set_index('Model')
print(comparison_df.to_string(float_format=lambda x: f'{x:.4f}'))

# Save results
comparison_df.to_csv('models/binary_comparison.csv')
print("\n💾 Saved to models/binary_comparison.csv")

In [ ]:
# 📊 Visual Comparison
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Metrics comparison
metrics_plot = ['Accuracy', 'F1', 'Precision', 'Recall', 'ROC-AUC']
x = np.arange(len(metrics_plot))
width = 0.2
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

for i, name in enumerate(model_names):
    vals = [comparison_df.loc[name, m] for m in metrics_plot]
    axes[0].bar(x + i*width, vals, width, label=name, color=colors[i], edgecolor='white')

axes[0].set_xticks(x + width*1.5)
axes[0].set_xticklabels(metrics_plot)
axes[0].set_ylim(90, 101)
axes[0].set_title('📊 Model Comparison — Metrics (%)', fontsize=14, fontweight='bold', color='white')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.2, axis='y')

# Training time comparison
axes[1].barh(model_names, train_times, color=colors, edgecolor='white', height=0.5)
for i, t in enumerate(train_times):
    axes[1].text(t + max(train_times)*0.02, i, f'{t:.1f}s', va='center', fontweight='bold', color='white')
axes[1].set_title('⏱️ Training Time (seconds)', fontsize=14, fontweight='bold', color='white')
axes[1].grid(True, alpha=0.2, axis='x')

plt.suptitle('Level 1: Binary Classification — Model Comparison', fontsize=16, fontweight='bold', color='#00D4AA', y=1.02)
plt.tight_layout()
plt.savefig('figures/binary_model_comparison.png', dpi=150, bbox_inches='tight', facecolor='#1a1a2e')
plt.show()

In [ ]:
# 📊 ROC Curves
fig, ax = plt.subplots(figsize=(10, 8))

for i, name in enumerate(model_names):
    fpr, tpr, _ = roc_curve(y_test, probabilities[name])
    roc_auc_val = auc(fpr, tpr)
    ax.plot(fpr, tpr, color=colors[i], linewidth=2.5, label=f'{name} (AUC={roc_auc_val:.4f})')

ax.plot([0, 1], [0, 1], 'w--', alpha=0.3, linewidth=1)
ax.set_xlabel('False Positive Rate', fontsize=13)
ax.set_ylabel('True Positive Rate', fontsize=13)
ax.set_title('📈 ROC Curves — Binary Classification', fontsize=16, fontweight='bold', color='#00D4AA')
ax.legend(fontsize=12, loc='lower right')
ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.savefig('figures/binary_roc_curves.png', dpi=150, bbox_inches='tight', facecolor='#1a1a2e')
plt.show()
print("💾 Saved to figures/binary_roc_curves.png")

In [ ]:
# 📊 Confusion Matrices for all 4 models
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

for ax, name in zip(axes.ravel(), model_names):
    cm = confusion_matrix(y_test, predictions[name])
    cm_pct = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    sns.heatmap(cm_pct, annot=True, fmt='.2f', cmap='RdYlGn',
                xticklabels=['Benign', 'Attack'], yticklabels=['Benign', 'Attack'],
                ax=ax, linewidths=2, linecolor='white',
                annot_kws={'fontsize': 14, 'fontweight': 'bold'})
    ax.set_title(f'{name}', fontsize=14, fontweight='bold', color='white')
    ax.set_xlabel('Predicted', fontsize=11)
    ax.set_ylabel('Actual', fontsize=11)

plt.suptitle('📊 Confusion Matrices — Binary Classification (% per class)', fontsize=16, fontweight='bold', color='#00D4AA', y=1.02)
plt.tight_layout()
plt.savefig('figures/binary_confusion_matrices.png', dpi=150, bbox_inches='tight', facecolor='#1a1a2e')
plt.show()
print("💾 Saved to figures/binary_confusion_matrices.png")

In [ ]:
# Classification Reports
for name in model_names:
    print(f"\n{'='*60}")
    print(f"📋 {name} — Classification Report")
    print(f"{'='*60}")
    print(classification_report(y_test, predictions[name], target_names=['Benign', 'Attack']))

In [ ]:
# Save all results
binary_results = {
    'timestamp': datetime.now().isoformat(),
    'level': 'Binary (Benign vs Attack)',
    'device': 'GPU (CUDA) for XGBoost & LightGBM',
    'models': {}
}
for name in model_names:
    y_p = predictions[name]
    y_pr = probabilities[name]
    cm = confusion_matrix(y_test, y_p)
    binary_results['models'][name] = {
        'accuracy': float(accuracy_score(y_test, y_p)),
        'f1': float(f1_score(y_test, y_p)),
        'precision': float(precision_score(y_test, y_p)),
        'recall': float(recall_score(y_test, y_p)),
        'roc_auc': float(roc_auc_score(y_test, y_pr)),
        'train_time': train_times[model_names.index(name)],
        'inference_time': infer_times[model_names.index(name)],
        'confusion_matrix': cm.tolist()
    }

with open('models/binary_results.json', 'w') as f:
    json.dump(binary_results, f, indent=2)

print("\n🏆" * 20)
print(f"\n  ✅ BINARY CLASSIFICATION COMPLETE!")
print(f"  📊 4 models trained and evaluated")
print(f"  🎮 XGBoost & LightGBM on GPU")
print(f"  💾 All results saved")
print(f"\n" + "🏆" * 20)